In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("kobe.csv")

In [3]:
data.head()

,action_type,combined_shot_type,game_id,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,game_date,matchup,opponent
0,Jump Shot,Jump Shot,20000012,10,1,0,Jan-00,27,18,0,2PT Field Goal,Right Side(R),Mid-Range,2000/10/31,LAL @ POR,POR
1,Jump Shot,Jump Shot,20000012,10,1,0,Jan-00,22,15,0,2PT Field Goal,Left Side(L),Mid-Range,2000/10/31,LAL @ POR,POR
2,Jump Shot,Jump Shot,20000012,7,1,0,Jan-00,45,16,0,2PT Field Goal,Left Side Center(LC),Mid-Range,2000/10/31,LAL @ POR,POR
3,Jump Shot,Jump Shot,20000012,6,1,0,Jan-00,52,22,0,2PT Field Goal,Right Side Center(RC),Mid-Range,2000/10/31,LAL @ POR,POR
4,Driving Dunk Shot,Dunk,20000012,6,2,0,Jan-00,19,0,0,2PT Field Goal,Center(C),Restricted Area,2000/10/31,LAL @ POR,POR


In [4]:
data.shape

(30697, 16)

In [5]:
print(len(data["shot_zone_basic"].unique()))
data["shot_zone_basic"].unique()

7


array(['Mid-Range', 'Restricted Area', 'In The Paint (Non-RA)',
       'Above the Break 3', 'Right Corner 3', 'Backcourt',
       'Left Corner 3'], dtype=object)

In [6]:
len(data["shot_distance"].unique())

74

In [7]:
#(1)
print("ANS: ",data.groupby(['shot_distance']).size().nlargest(1).index.values[0])

ANS:  0


In [8]:
#(2)
print("ANS: ",data[data["shot_made_flag"]==1].groupby(['shot_zone_area']).size().nlargest(1).index.values[0])

ANS:  Center(C)


In [9]:
#(3)
print("ANS: ")
print("2PT Field Goal: ",round(len(data[data["opponent"]=="SAS"][data["shot_made_flag"]==1][data["shot_type"]=="2PT Field Goal"])/len(data[data["opponent"]=="SAS"][data["shot_type"]=="2PT Field Goal"]),3))
print("3PT Field Goal: ",round(len(data[data["opponent"]=="SAS"][data["shot_made_flag"]==1][data["shot_type"]=="3PT Field Goal"])/len(data[data["opponent"]=="SAS"][data["shot_type"]=="3PT Field Goal"]),3))

ANS: 
2PT Field Goal:  0.384
3PT Field Goal:  0.256


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [10]:
#(4)
print("ANS: \n", data.groupby(["game_id","opponent"]).mean().reset_index()[["game_id","opponent"]].groupby(["opponent"]).size().nlargest())

ANS: 
 opponent
SAS    91
PHX    87
UTA    84
DEN    83
POR    81
dtype: int64


In [11]:
#(5)
data["shot_type_score"]=data["shot_type"].str.get(0).astype(int)
print("ANS: ")
round(data[data["shot_made_flag"]==1].groupby(["opponent"])["shot_type_score"].sum()/data.groupby(["game_id","opponent"]).mean().reset_index()[["game_id","opponent"]].groupby(["opponent"]).size(),2).nlargest()

ANS: 


opponent
PHX    17.71
MEM    17.43
SAC    17.36
NYK    17.21
GSW    17.13
dtype: float64

In [12]:
#(6)
print("ANS: ")
data[data["minutes_remaining"]<=2][data["shot_made_flag"]==1].groupby(["game_id"])["shot_type_score"].sum().nlargest()

ANS: 


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


game_id
20900706    19
21200906    18
40500156    18
20500471    17
21400056    17
Name: shot_type_score, dtype: int64

In [13]:
#(7)
data0=data[data["playoffs"]==0][data["minutes_remaining"]==0][data["seconds_remaining"]<=30][data["action_type"]=="Jump Shot"]
data0["game_date_year"]=data0["game_date"].str.split("/").str.get(0)
print("ANS: "); round(data0[data0["shot_made_flag"]==1].groupby(["game_date_year"]).size()/data0.groupby(["game_date_year"]).size(),3)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


ANS: 


game_date_year
1996    0.429
1997    0.240
1998    0.171
1999    0.232
2000    0.211
2001    0.190
2002    0.186
2003    0.163
2004    0.181
2005    0.216
2006    0.183
2007    0.284
2008    0.130
2009    0.148
2010    0.188
2011    0.109
2012    0.217
2013    0.196
2014    0.038
2015    0.176
2016    0.059
dtype: float64

In [14]:
#(8)
data0=data[data["playoffs"]==0][data["shot_type"]=="2PT Field Goal"] #正規賽
data0=(data0[data0["shot_made_flag"]==1].groupby(["opponent"]).size()/data0.groupby(["opponent"]).size()).reset_index()
data1=data[data["playoffs"]==1][data["shot_type"]=="2PT Field Goal"] #季後賽
data1=(data1[data1["shot_made_flag"]==1].groupby(["opponent"]).size()/data1.groupby(["opponent"]).size()).reset_index()
data2=pd.merge(data1,data0,on="opponent")
data2["score_difference"]=round(data2["0_x"]-data2["0_y"],3)
data2[["opponent","score_difference"]][data2["score_difference"]>0].nlargest(data2[data2["score_difference"]>0].shape[0],"score_difference")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,opponent,score_difference
7,NJN,0.054
15,SAS,0.044
17,UTA,0.033
9,OKC,0.021
1,DAL,0.018
12,PHX,0.016
2,DEN,0.009
8,NOH,0.009
10,ORL,0.008
0,BOS,0.005


In [15]:
#(9)
data3=(data[data["shot_made_flag"]==1].groupby(["game_date"])["shot_type_score"].sum()).reset_index()
data3["game_datetime"]=pd.to_datetime(data3["game_date"])
data3.sort_values('game_datetime', inplace=True)
data3["row_index"]=np.arange(0, data3.shape[0])
data3=data3[data3["shot_type_score"]>=25]

In [16]:
data3["rank"]=0
count=0
for i in range(data3.shape[0]-1):
    if data3["row_index"].iloc[i+1]-data3["row_index"].iloc[i]!=1: count+=1
    data3["rank"].iloc[i+1]=count

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
data3.groupby(["rank"]).size().nlargest()

rank
26    5
61    5
75    5
7     4
46    3
dtype: int64

In [18]:
#(10)
dataRegular=data[data["playoffs"]==0]
dataPlayoffs=data[data["playoffs"]==1]

In [19]:
for i in dataRegular["combined_shot_type"].unique():
    print("Regular +", i)
    data_i=dataRegular[dataRegular["combined_shot_type"]==i]
    print(round(data_i[data_i["shot_made_flag"]==1].groupby(["opponent"])["shot_type_score"].sum()/data_i.groupby(["game_id","opponent"]).mean().reset_index()[["game_id","opponent"]].groupby(["opponent"]).size(),4).nlargest(),"\n")

Regular + Jump Shot
opponent
NOP    13.0625
NYK    12.8529
HOU    12.5410
POR    12.0492
SEA    11.8810
dtype: float64 

Regular + Dunk
opponent
BKN    4.0000
TOR    3.7333
DET    3.2000
MEM    3.1429
CHI    3.1250
dtype: float64 

Regular + Layup
opponent
POR    4.5517
MEM    4.2128
ATL    4.1600
GSW    4.1212
PHX    4.1053
dtype: float64 

Regular + Tip Shot
opponent
BKN    2.0000
BOS    2.0000
NOH    2.0000
NOP    2.0000
MIL    1.3333
dtype: float64 

Regular + Hook Shot
opponent
BKN    2.0
HOU    2.0
MEM    2.0
NOH    2.0
NOP    2.0
dtype: float64 

Regular + Bank Shot
opponent
WAS    3.0
ATL    2.0
BOS    2.0
CHA    2.0
IND    2.0
dtype: float64 



In [20]:
for i in dataPlayoffs["combined_shot_type"].unique():
    print("Playoffs +", i)
    data_i=dataPlayoffs[dataPlayoffs["combined_shot_type"]==i]
    print(round(data_i[data_i["shot_made_flag"]==1].groupby(["opponent"])["shot_type_score"].sum()/data_i.groupby(["game_id","opponent"]).mean().reset_index()[["game_id","opponent"]].groupby(["opponent"]).size(),4).nlargest(),"\n")

Playoffs + Jump Shot
opponent
ORL    17.4000
DAL    15.2500
PHX    14.3043
BOS    14.0000
SAC    13.6875
dtype: float64 

Playoffs + Layup
opponent
SAS    4.9655
SAC    4.2500
DEN    4.1176
DET    4.0000
MIN    4.0000
dtype: float64 

Playoffs + Tip Shot
opponent
HOU    2.0
PHI    2.0
PHX    2.0
POR    2.0
IND    1.0
dtype: float64 

Playoffs + Dunk
opponent
OKC    6.0000
DET    3.5000
SAS    3.0588
MIN    2.8571
NOH    2.6667
dtype: float64 

Playoffs + Hook Shot
opponent
SAS    4.0
DAL    2.0
DEN    2.0
HOU    2.0
NOH    2.0
dtype: float64 

Playoffs + Bank Shot
opponent
HOU    4.0
BOS    3.0
DEN    2.5
NOH    2.0
OKC    2.0
dtype: float64 

